***Preprocessing London Weather data for using later

Import pySpark

In [1]:
# import pyspark
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DP_weather').getOrCreate()

Reading three years london weather data in to dataframe

In [2]:
# Importing data which has a header. Schema is automatically configured.
df17 = spark.read.csv('Datasets/london weather 2017.csv', header=True, inferSchema=True)
df18 = spark.read.csv('Datasets/london weather 2018.csv', header=True, inferSchema=True)
df19 = spark.read.csv('Datasets/london weather 2019.csv', header=True, inferSchema=True)

Merging into one single weather dataset

In [3]:
df_london_weather = df17.union(df18).union(df19)

In [7]:
df_london_weather.printSchema()

root
 |-- date: date (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipcover: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- day_length: double (nullable = true)



Import functions

In [4]:
from pyspark.sql.functions import *

For safty, people may not like riding bicycle in dark
Calculate hours different between sunset and sunrise
Round the number to one decimal

In [5]:
df_london_weather = df_london_weather.withColumn('day_length', bround((unix_timestamp("sunset")-unix_timestamp("sunrise"))/3600,1))

Weather condition may not impact riding bicycles
Drop columns from weather dataset

In [6]:
#Select columns we later
df_london_weather = df_london_weather.select('datetime', 'feelslike','precip','precipcover','windspeed','day_length')
#rename the column datetime to date
df_london_weather = df_london_weather.withColumnRenamed('datetime', 'date')

**Write processed dataset to CSV for use later

In [ ]:
df_london_weather.coalesce(1).write.format('com.databricks.spark.csv').save('DataSets/weather.csv',header = 'true')